##### A stemmed CountVectorizer

In [1]:
import copy
from sklearn.feature_extraction.text import CountVectorizer

# Overide CountVectorizer to integrate stemming
class StemmedCountVectorizer(CountVectorizer):
    def __init__(self, stemmer, **kwargs):
        super(StemmedCountVectorizer, self).__init__(**kwargs)
        self.stemmer = stemmer

    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (self.stemmer.stem(w) for w in analyzer(doc))

    def get_params(self, deep=True):
        params = super().get_params(deep=deep)
        cp = copy.copy(self)
        cp.__class__ = CountVectorizer
        params.update(CountVectorizer.get_params(cp, deep))
        return params

In [12]:
# -*- coding: utf-8 -*-

"""FakeNewsClassifier Trainer

This script conduct the following steps (overview):

1. Load the processes dataset from {PROJECT_DIR}/data/processed as dataframe
2. Concatenate title and text in one column
3. Split the dataset into train and test

4. Set up a sklearn pipeline with preprocessor and classifier for the combined column (title+text)
   and perform hyperparam search 
5. Extract best score and best_params for and log it
6. Predict on testset and log the score

7. Pickle best pipe to disk (final model)

"""
# %%
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

import os
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, recall_score, precision_score
from dotenv import find_dotenv, load_dotenv
import nltk
from nltk.stem.snowball import GermanStemmer
from sklearn.feature_extraction.text import CountVectorizer
import dagshub
from joblib import dump


load_dotenv(find_dotenv())
#%% Load and split dataset
# INPUTFILE = os.path.join(os.getenv('PROJECT_DIR'), 'data', 'processed', 'fake_news_processed.csv')
INPUTFILE = os.path.join('data', 'datasets_merged.csv')
df = pd.read_csv(INPUTFILE, sep=';')
logger.info('Distribution of fake news in entire dataset: \n%s' % df.fake.value_counts(normalize=True))

X = df['title']+' '+df['text']
y = df['fake']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                    test_size=0.2,
                    stratify=y,
                    random_state=42)

# Download and init stopwordlist
nltk.download('stopwords')
STOPWORD_LIST = nltk.corpus.stopwords.words('german')

#print(X_train[100])

# %% Test with CountVectorizer
"""
count_vec = StemmedCountVectorizer(token_pattern=r'\b[a-zA-Z]{2,}\b',
                                max_features=100,
                                stop_words=STOPWORD_LIST,
                                lowercase=True,
                                stemmer=GermanStemmer(ignore_stopwords=True),
                                ngram_range=(2,2)
                        )

word_counts = count_vec.fit_transform(X_train).todense()

df_body = pd.DataFrame(word_counts, columns=count_vec.get_feature_names())
df_body_trans = pd.DataFrame(df_body.T.sum(axis=1), columns=['count'])
df_body_trans.sort_values(by='count', ascending=False).head(20)

"""

#%% Define param grid for tuning
param_grid = {'vectorizer__max_features':[500],
        'clf__n_estimators': [400],
        'clf__learning_rate': [0.9],
        'clf__max_depth': [5]}

# Constants for training
# MODEL_PATH = os.getenv('MODEL_PATH')
MODEL_PATH = 'model.pkl'
CV_SCORING = 'f1'
CV_FOLDS = 3

# %%
with dagshub.dagshub_logger() as dagslog:
        # Pipeline
        pipe = Pipeline(steps=[
        ('vectorizer', StemmedCountVectorizer(stop_words=STOPWORD_LIST,
                                   token_pattern=r'\b[a-zA-Z]{2,}\b',
                                   stemmer=GermanStemmer(ignore_stopwords=True),
                                   lowercase=True)),
        ('clf', CatBoostClassifier(allow_writing_files=False))
        ])
        
        # pipe_title_tune.get_params().keys()
        logger.info('Start training estimator pipeline')
        cv_grid = GridSearchCV(pipe, param_grid, scoring=CV_SCORING, cv=CV_FOLDS, n_jobs=2)
        cv_grid.fit(X_train, y_train)
        #cv_grid.best_estimator_.named_steps['clf'].get_all_params()

        # Log and assign best score and params to var
        logger.info('Best params from GridSearchCV: %s' % str(cv_grid.best_params_))
        dagslog.log_hyperparams({'model_path': MODEL_PATH})
        dagslog.log_hyperparams({'cv_score': CV_SCORING})
        dagslog.log_hyperparams({'best_cv_params': cv_grid.best_params_})

        logger.info('Best %s from GridSearchCV: %s' % (CV_SCORING, str(cv_grid.best_score_)))
        dagslog.log_metrics({'best_cv_score': cv_grid.best_score_})
        
        # Predict on testdata
        y_pred = cv_grid.best_estimator_.predict(X_test)
        logger.info('f1_score for testdata: %s' % str(f1_score(y_test, y_pred)))
        dagslog.log_metrics({'f1_score_on_testdata': f1_score(y_test, y_pred)})
        logger.info('precision_score for testdata: %s' % str(precision_score(y_test, y_pred)))
        dagslog.log_metrics({'precision_score_on_testdata': precision_score(y_test, y_pred)})
        logger.info('recall_score for testdata: %s' % str(recall_score(y_test, y_pred)))
        dagslog.log_metrics({'recall_score_on_testdata': recall_score(y_test, y_pred)})
        logger.info('Classification report for testdata: \n%s' % classification_report(y_test, y_pred))
        logger.info('Confusion matrix for testdata: \n%s' % confusion_matrix(y_test, y_pred))

        # Dump model to disk
        dump(cv_grid.best_estimator_, MODEL_PATH)

INFO:root:Distribution of fake news in entire dataset: 
0    0.922928
1    0.077072
Name: fake, dtype: float64
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fjun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
INFO:root:Start training estimator pipeline


0:	learn: 0.1790363	total: 43.9ms	remaining: 17.5s
1:	learn: 0.1292465	total: 58.6ms	remaining: 11.7s
2:	learn: 0.1143598	total: 73.4ms	remaining: 9.71s
3:	learn: 0.1028559	total: 87ms	remaining: 8.61s
4:	learn: 0.0949349	total: 101ms	remaining: 8.01s
5:	learn: 0.0891569	total: 114ms	remaining: 7.51s
6:	learn: 0.0860463	total: 128ms	remaining: 7.19s
7:	learn: 0.0828056	total: 141ms	remaining: 6.9s
8:	learn: 0.0804890	total: 155ms	remaining: 6.73s
9:	learn: 0.0802443	total: 166ms	remaining: 6.47s
10:	learn: 0.0773814	total: 179ms	remaining: 6.33s
11:	learn: 0.0743589	total: 193ms	remaining: 6.24s
12:	learn: 0.0742056	total: 205ms	remaining: 6.11s
13:	learn: 0.0712867	total: 219ms	remaining: 6.04s
14:	learn: 0.0710549	total: 238ms	remaining: 6.11s
15:	learn: 0.0693764	total: 252ms	remaining: 6.04s
16:	learn: 0.0692207	total: 264ms	remaining: 5.96s
17:	learn: 0.0672085	total: 280ms	remaining: 5.94s
18:	learn: 0.0662156	total: 294ms	remaining: 5.89s
19:	learn: 0.0660709	total: 306ms	remain

173:	learn: 0.0276435	total: 2.31s	remaining: 2.99s
174:	learn: 0.0272859	total: 2.32s	remaining: 2.98s
175:	learn: 0.0272140	total: 2.33s	remaining: 2.97s
176:	learn: 0.0272094	total: 2.34s	remaining: 2.95s
177:	learn: 0.0270789	total: 2.36s	remaining: 2.94s
178:	learn: 0.0270125	total: 2.37s	remaining: 2.93s
179:	learn: 0.0269998	total: 2.38s	remaining: 2.91s
180:	learn: 0.0269821	total: 2.4s	remaining: 2.9s
181:	learn: 0.0269769	total: 2.41s	remaining: 2.88s
182:	learn: 0.0268652	total: 2.42s	remaining: 2.87s
183:	learn: 0.0266784	total: 2.43s	remaining: 2.86s
184:	learn: 0.0266743	total: 2.44s	remaining: 2.84s
185:	learn: 0.0265352	total: 2.46s	remaining: 2.83s
186:	learn: 0.0265061	total: 2.47s	remaining: 2.81s
187:	learn: 0.0265027	total: 2.48s	remaining: 2.8s
188:	learn: 0.0264620	total: 2.5s	remaining: 2.79s
189:	learn: 0.0263202	total: 2.51s	remaining: 2.77s
190:	learn: 0.0263165	total: 2.52s	remaining: 2.76s
191:	learn: 0.0261984	total: 2.53s	remaining: 2.74s
192:	learn: 0.02

332:	learn: 0.0173744	total: 4.34s	remaining: 874ms
333:	learn: 0.0173729	total: 4.36s	remaining: 861ms
334:	learn: 0.0172831	total: 4.37s	remaining: 848ms
335:	learn: 0.0172528	total: 4.38s	remaining: 835ms
336:	learn: 0.0170778	total: 4.4s	remaining: 822ms
337:	learn: 0.0170558	total: 4.41s	remaining: 809ms
338:	learn: 0.0169999	total: 4.42s	remaining: 796ms
339:	learn: 0.0169425	total: 4.44s	remaining: 783ms
340:	learn: 0.0168844	total: 4.45s	remaining: 770ms
341:	learn: 0.0168831	total: 4.46s	remaining: 756ms
342:	learn: 0.0168805	total: 4.47s	remaining: 743ms
343:	learn: 0.0168684	total: 4.48s	remaining: 730ms
344:	learn: 0.0168546	total: 4.5s	remaining: 717ms
345:	learn: 0.0167984	total: 4.51s	remaining: 704ms
346:	learn: 0.0167353	total: 4.53s	remaining: 691ms
347:	learn: 0.0167231	total: 4.54s	remaining: 678ms
348:	learn: 0.0166949	total: 4.55s	remaining: 665ms
349:	learn: 0.0166856	total: 4.56s	remaining: 652ms
350:	learn: 0.0166549	total: 4.58s	remaining: 639ms
351:	learn: 0.

INFO:root:Best params from GridSearchCV: {'clf__learning_rate': 0.9, 'clf__max_depth': 5, 'clf__n_estimators': 400, 'vectorizer__max_features': 500}
INFO:root:Best f1 from GridSearchCV: 0.8555504998353983


390:	learn: 0.0150010	total: 5.08s	remaining: 117ms
391:	learn: 0.0149992	total: 5.1s	remaining: 104ms
392:	learn: 0.0149916	total: 5.11s	remaining: 91ms
393:	learn: 0.0149502	total: 5.12s	remaining: 78ms
394:	learn: 0.0149196	total: 5.13s	remaining: 65ms
395:	learn: 0.0148455	total: 5.15s	remaining: 52ms
396:	learn: 0.0148343	total: 5.16s	remaining: 39ms
397:	learn: 0.0147797	total: 5.17s	remaining: 26ms
398:	learn: 0.0147034	total: 5.19s	remaining: 13ms
399:	learn: 0.0146892	total: 5.2s	remaining: 0us


INFO:root:f1_score for testdata: 0.8681434599156118
INFO:root:precision_score for testdata: 0.8878101402373247
INFO:root:recall_score for testdata: 0.849329205366357
INFO:root:Classification report for testdata: 
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     11599
           1       0.89      0.85      0.87       969

    accuracy                           0.98     12568
   macro avg       0.94      0.92      0.93     12568
weighted avg       0.98      0.98      0.98     12568

INFO:root:Confusion matrix for testdata: 
[[11495   104]
 [  146   823]]


In [9]:
key = 'HOME'
value = os.getenv(key)
  
# Print the value of 'HOME'
# environment variable
print("Value of 'HOME' environment variable :", value) 
  
# Get the value of 'JAVA_HOME'
# environment variable
key = 'MODEL_PATH'
value = os.getenv(key)
  
# Print the value of 'JAVA_HOME'
# environment variable
print("Value of 'JAVA_HOME' environment variable :", value) 

Value of 'HOME' environment variable : None
Value of 'JAVA_HOME' environment variable : None
